# House Loan Data Analysis

**Problem Statement:**
For a safe and secure lending experience, it's important to analyze the past data. In this project, you have to build a deep learning model to predict the chance of default for future loans using the historical data. As you will see, this dataset is highly imbalanced and includes a lot of features that make this problem more challenging.

**Objective:** Create a model that predicts whether or not an applicant will be able to repay a loan using the historical data

**Domain:** Finance

**Analysis to be done:** Perform data preprocessing and build a deep learning prediction model

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tabulate import tabulate

## Data Analysis

This section analyzis the data available. First step is to analuze all 122 columns for:
1. Categorization: numeric and categorical
2. Columns with null values: 
3. Imputation strategies

**Read Data:**

In [2]:
data = pd.read_csv("./data/project/loan_data.csv")
data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
data.shape

(307511, 122)

In [4]:
def get_numeric_columns(data):
    return data.select_dtypes(include=['number']).columns

def get_categorical_columns(data):
    return data.select_dtypes(include=['object']).columns

numeric_columns = get_numeric_columns(data)
categorical_columns = get_categorical_columns(data)

print(f"Numeric columns: {list(numeric_columns)}")
print(f"Numeric columns: {len(numeric_columns)}")

Numeric columns: ['SK_ID_CURR', 'TARGET', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATA

In [5]:
print(f"Categorical columns: {list(categorical_columns)}")
print(f"Categorical columns: {len(categorical_columns)}")

Categorical columns: ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']
Categorical columns: 16


In [6]:
descriptions = pd.read_csv("./data/project/load_data_dictionary.csv")
print(len(descriptions))
variable_descriptions = {}
for index, row in descriptions.iterrows():
    variable_descriptions[row['Variable Name']] = row['Description']

122


In [7]:
# Missing Values
null_data = data.isnull().sum()
null_data_df = pd.DataFrame(null_data, columns=['null_count'])
null_data_df['null_percentage'] = null_data_df['null_count'] * 100 / len(data)
# Add descriptions to null_data_df using map, the index name is key value in variable_descriptions
null_data_df['description'] = null_data_df.index.map(variable_descriptions)

In [8]:
print(tabulate(null_data_df, headers='keys', tablefmt='plain'))

                                null_count    null_percentage  description
SK_ID_CURR                               0        0            Unique identifier for each client.
TARGET                                   0        0            Whether the loan was repaid (1 for yes, 0 for no).
NAME_CONTRACT_TYPE                       0        0            Type of loan contract (e.g., Cash loans, Revolving loans).
CODE_GENDER                              0        0            Gender of the applicant.
FLAG_OWN_CAR                             0        0            Whether the applicant owns a car (Yes/No).
FLAG_OWN_REALTY                          0        0            Whether the applicant owns a house or flat (Yes/No).
CNT_CHILDREN                             0        0            Number of children the applicant has.
AMT_INCOME_TOTAL                         0        0            Total income of the applicant.
AMT_CREDIT                               0        0            Total credit amount of 